# 连续词袋模型 (CBOW) 实现

本 Notebook 演示如何使用 Python 和 TensorFlow/Keras 构建和训练一个简单的连续词袋模型 (CBOW)。

**目标:**

*   理解 CBOW 模型的基本原理。
*   使用 TensorFlow/Keras 实现 CBOW 模型。
*   准备训练数据。
*   训练模型并评估其性能。

## 1. 导入必要的库

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import skipgrams

## 2. 准备数据

首先，我们需要准备用于训练的数据。 在这个例子中，我们使用一个简单的句子。

**注意:** 实际应用中，你需要使用更大的语料库。

In [2]:
sentence = "The quick brown fox jumps over the lazy dog."

# 1. 数据预处理
tokenizer = Tokenizer()
tokenizer.fit_on_texts([sentence])

word_index = tokenizer.word_index
vocabulary_size = len(word_index) + 1 # +1 for the padding token (if needed)

print(f"词汇表大小: {vocabulary_size}")
print(f"单词索引: {word_index}")

encoded_sentence = tokenizer.texts_to_sequences([sentence])[0]

## 3. 创建训练数据 (Skip-grams)

CBOW 通过上下文词预测目标词。我们需要生成上下文-目标词对。 这里使用 `skipgrams` 函数，它生成 skip-gram 形式的词对，适用于 CBOW 和 Skip-gram 模型。

In [3]:
# 2. 生成 Skip-gram 数据
window_size = 2 # 上下文窗口大小
negative_samples = 1.0 # 每个正样本的负样本数

couples, labels = skipgrams(encoded_sentence, vocabulary_size, window_size=window_size, negative_samples=negative_samples)

word_target, word_context = zip(*couples)
word_target = np.array(word_target, dtype="int32")
word_context = np.array(word_context, dtype="int32")

print(f"生成的词对数量: {len(couples)}")
print(f"第一个词对: target={word_target[0]}, context={word_context[0]}, label={labels[0]}")

## 4. 构建 CBOW 模型

In [4]:
# 3. 构建 CBOW 模型
embedding_dim = 50 # 词嵌入维度

cbow = keras.models.Sequential([
    layers.Embedding(vocabulary_size, embedding_dim, input_length=1),
    layers.Reshape((embedding_dim,)),
    layers.Dense(vocabulary_size, activation='sigmoid') # 使用 softmax 进行多分类, 优化器需要调整
])

cbow.summary()

## 5. 编译和训练模型

In [5]:
# 4. 编译模型
cbow.compile(optimizer='adam', # Adam 优化器
              loss='sparse_categorical_crossentropy', # 适用于多分类问题
              metrics=['accuracy'])

# 5. 训练模型
epochs = 100

cbow.fit([word_target, word_context], np.array(labels), epochs=epochs, batch_size=256, verbose=0)

print("训练完成!")

## 6.  评估 (可选) 和使用词嵌入

由于数据集非常小，直接评估模型的质量意义不大。  通常，你需要在一个更大的数据集上训练模型，然后评估生成的词嵌入在其他任务上的表现 (例如，词语相似度，文本分类等)。

这里展示如何提取词嵌入。


In [6]:
# 6. 获取词嵌入
embedding_layer = cbow.layers[0]
embeddings = embedding_layer.get_weights()[0]

print(f"词嵌入矩阵的形状: {embeddings.shape}") # (vocabulary_size, embedding_dim)

# 获取 "fox" 的词嵌入
word = "fox"
word_id = word_index[word]
embedding_vector = embeddings[word_id]

print(f"'{word}' 的词嵌入: {embedding_vector}")

## 7. 完整代码（整合）

为了方便复制，这里提供一个完整的代码版本。

```python
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import skipgrams

sentence = "The quick brown fox jumps over the lazy dog."

# 1. 数据预处理
tokenizer = Tokenizer()
tokenizer.fit_on_texts([sentence])

word_index = tokenizer.word_index
vocabulary_size = len(word_index) + 1 # +1 for the padding token (if needed)

print(f"词汇表大小: {vocabulary_size}")
print(f"单词索引: {word_index}")

encoded_sentence = tokenizer.texts_to_sequences([sentence])[0]

# 2. 生成 Skip-gram 数据
window_size = 2 # 上下文窗口大小
negative_samples = 1.0 # 每个正样本的负样本数

couples, labels = skipgrams(encoded_sentence, vocabulary_size, window_size=window_size, negative_samples=negative_samples)

word_target, word_context = zip(*couples)
word_target = np.array(word_target, dtype="int32")
word_context = np.array(word_context, dtype="int32")

print(f"生成的词对数量: {len(couples)}")
print(f"第一个词对: target={word_target[0]}, context={word_context[0]}, label={labels[0]}")

# 3. 构建 CBOW 模型
embedding_dim = 50 # 词嵌入维度

cbow = keras.models.Sequential([
    layers.Embedding(vocabulary_size, embedding_dim, input_length=1),
    layers.Reshape((embedding_dim,)),
    layers.Dense(vocabulary_size, activation='sigmoid') # 使用 softmax 进行多分类, 优化器需要调整
])

cbow.summary()

# 4. 编译模型
cbow.compile(optimizer='adam', # Adam 优化器
              loss='sparse_categorical_crossentropy', # 适用于多分类问题
              metrics=['accuracy'])

# 5. 训练模型
epochs = 100

cbow.fit([word_target, word_context], np.array(labels), epochs=epochs, batch_size=256, verbose=0)

print("训练完成!")

# 6. 获取词嵌入
embedding_layer = cbow.layers[0]
embeddings = embedding_layer.get_weights()[0]

print(f"词嵌入矩阵的形状: {embeddings.shape}") # (vocabulary_size, embedding_dim)

# 获取 "fox" 的词嵌入
word = "fox"
word_id = word_index[word]
embedding_vector = embeddings[word_id]

print(f"'{word}' 的词嵌入: {embedding_vector}")
```

## 总结

本 Notebook 演示了如何使用 TensorFlow/Keras 构建和训练一个简单的 CBOW 模型。 请记住，这只是一个基础示例，实际应用中需要更大的数据集和更复杂的模型结构。